# Today's topic: How Spark Parititions influence saving data with parquet

# 0. Set-Ups

General hints for this notebook:
- Spark UI usually accesible by http://localhost:4040/ or http://localhost:4041/
- Deep dive Spark UI happens in later episodes
- sc.setJobDescription("Description") replaces the Job Description of an action in the Spark UI with your own
- sdf.rdd.getNumPartitions() returns the number partitions of the current Spark DataFrame
- sdf.write.format("noop").mode("overwrite").save() is a good way to analyze and initiate actions for transformations without side effects during an actual write

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import gresearch.spark.parquet
import math
import time


In [3]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: 
"""
sc = spark.sparkContext

In [4]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache datafrimes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [5]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [6]:
def rows_per_partition(sdf: "DataFrame") -> None:
    num_rows = sdf.count()
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id").count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id").show()

In [7]:
def rows_per_partition_col(sdf: "DataFrame", num_rows: int, col: str) -> None:
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id", col).count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id", col).show()


In [8]:
BASE_DIR = "D:/Spark/Data"
results_dict = {}
results_list = []

In [9]:
def write_generator(num_rows, num_files):
    sdf = sdf_generator(num_rows, num_files)
    path = f"{BASE_DIR}/{num_files}_files_{num_rows}_rows.parquet"
    sc.setJobDescription(f"Write {num_files} files, {num_rows} rows")
    sdf.write.format("parquet").mode("overwrite").save(path)
    sc.setJobDescription("None")
    print(f"Num partitions written: {sdf.rdd.getNumPartitions()}")
    print(f"Saved Path: {path}")
    return path

In [38]:
def set_configs(maxPartitionsMB = 128, openCostInMB = 4, minPartitions = 4):
    maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
    openCostInBytes = math.ceil(openCostInMB*1024*1024)
    spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")
    spark.conf.set("spark.sql.files.openCostInBytes", str(openCostInBytes)+"b")
    spark.conf.set("spark.sql.files.minPartitionNum", str(minPartitions))
    print(" ")
    print("******** SPARK CONFIGURATIONS ********")
    print(f"MaxPartitionSize {maxPartitionsMB} MB or {maxPartitionsBytes} bytes")
    print(f"OpenCostInBytes {openCostInMB} MB or {openCostInBytes} bytes")
    print(f"Min Partitions: {minPartitions}")

    results_dict["maxPartitionsBytes"] = maxPartitionsMB

In [11]:
def get_parquet_meta_data(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .withColumn("calcNumBlocks", f.col("compressedMB")/128)
    )
    sdf.show(20, truncate=False)

In [12]:
def get_parquet_blocks(path):
    sdf = (
        spark.read.parquet_blocks(path)
        .dropDuplicates(["filename","block"])
        .orderBy("filename", "block")
        .withColumn("blockEnd", f.col("blockStart") + f.col("compressedBytes") - 1)
        .withColumn("blockMiddle", f.col("blockStart") + 0.5 * f.col("compressedBytes"))
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("filename", "block", "blockStart", "blockEnd", "blockMiddle", "compressedBytes", "compressedMB", "rows")
    )

    sdf.show(20, truncate=False)

In [13]:
def get_spark_partitions(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "start", "end", "length", "blocks", "compressedBytes", "compressedMB", "rows", "filename")
    )

    sdf.show(20, truncate=False)

In [14]:
def get_parquet_window_length(path):
    sdf = spark.read.parquet_partitions(path)
    val = sdf.select(f.max(sdf["length"]))
    max_length = val.collect()[0][0]
    print(f"Max Parquet window length: {round(max_length/1024/1024, 1)} MB or {max_length} bytes")

In [15]:
def get_parquet_file_size(path):
    sdf = (
        spark.read.parquet_metadata(path)
        .select("filename", "blocks", "compressedBytes", "rows")
        .dropDuplicates(["filename"])
    )
    sum = sdf.select(f.sum(sdf["compressedBytes"]))
    size = sum.collect()[0][0]
    return size

In [16]:
def round_half_up(n, decimals=0):
    multiplier = 10**decimals
    return math.floor(n * multiplier + 0.5) / multiplier

#source: https://realpython.com/python-rounding/

In [64]:
def estimate_num_partitions(file_size, num_files):
    #get spark values
    maxPartitionBytes = int(spark.conf.get("spark.sql.files.maxPartitionBytes")[:-1])
    openCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes")[:-1])
    minPartitionNum = int(spark.conf.get("spark.sql.files.minPartitionNum"))
    #Calculate maxSpliPartitionBytes
    # a) If we have bigger files bytesPerCorePadded will be bigger then openCostInBytes but also maxPartitionBytes. 
    # In this case we would limit the size as mayPartitionBytes will be the result of maxSplitPartitionBytes. e.g. 1 GB dataset, 4 cores, maxPartitions 128 MB
    # b) If bytesPerCorePadded is the result of maxSplitPartitionBytes we have a fair split of the data over all cores, e.g. 1 GB dataset, 4 cores, maxPartitions 300 MB
    # c) If bytesPerCore is to small we want to limit amount of Partitions to be opened. This is the cost here.
    paddedFileSize = file_size + num_files * openCostInBytes
    bytesPerCorePadded = paddedFileSize / minPartitionNum
    maxSplitPartitionBytes = min(maxPartitionBytes, max(openCostInBytes, bytesPerCorePadded))
    #Estimation of partitions from Internet
    estimated_num_partitions_int = paddedFileSize/maxSplitPartitionBytes
    #Own Estimator
    avg_file_size_padded = paddedFileSize/num_files
    bytesPerCore = file_size / minPartitionNum
    #Calculate number of files fitting into one partitions. Then calculate the number of partitions
    files_per_partition = max(1, math.floor(maxSplitPartitionBytes/avg_file_size_padded))
    estimated_num_partitions = num_files/files_per_partition
    print(" ")
    print("******** ESTIMATION OF MAX SPLIT PARTITION BYTES AND NO PARTITIONS ********")
    print(f"Avg file Size Padded: {round(avg_file_size_padded/1024/1024, 1)} MB or {avg_file_size_padded} bytes")
    print(f"Padded File Size: {round(paddedFileSize/1024/1024, 1)} MB or {paddedFileSize} bytes")
    print(f"SizePerCore: {round(bytesPerCore/1024/1024, 1)} MB or {bytesPerCore} bytes")
    print(f"SizePerCorePadded: {round(bytesPerCorePadded/1024/1024, 1)} MB or {bytesPerCorePadded} bytes")
    print(f"MaxSplitPartitionBytes: {round(maxSplitPartitionBytes/1024/1024, 1)} MB or {maxSplitPartitionBytes} bytes")
    print(f"MaxFilesPerPartition {files_per_partition}")
    print(f"EstimatedPartitions: {math.ceil(estimated_num_partitions)}, unrounded: {estimated_num_partitions}")
    print(f"EstimatedPartitionsInternet: {math.ceil(estimated_num_partitions_int)}, unrounded: {estimated_num_partitions_int}")

    results_dict["paddedFileSize"] = round(paddedFileSize/1024/1024, 1)
    results_dict["MBPerCore"] = round(bytesPerCore/1024/1024, 1)
    results_dict["MBPerCorePadded"] = round(bytesPerCorePadded/1024/1024, 1)
    results_dict["maxSplitPartitionBytes"] = round(maxSplitPartitionBytes/1024/1024, 1)
    results_dict["avg_file_size_padded"] = round(avg_file_size_padded/1024/1024, 1)
    results_dict["Maxfiles_per_partition"] = files_per_partition
    results_dict["MyEstimationPartitions"] = math.ceil(estimated_num_partitions)
    results_dict["InternetEstimationPartitions"] = math.ceil(estimated_num_partitions_int)
    


In [19]:
def bytes_rows_per_partition(path):
    sdf = (
        spark.read.parquet_partitions(path)
        .groupBy("partition").agg(f.sum("compressedBytes"), f.sum("rows"), f.count("partition"))
        .withColumnRenamed("sum(compressedBytes)", "compressedBytes")
        .withColumnRenamed("sum(rows)", "rows")
        .withColumnRenamed("count(partition)", "numFiles")
        .withColumn("compressedMB", f.round(f.col("compressedBytes")/1024/1024, 1))
        .select("partition", "numFiles", "compressedBytes","compressedMB","rows")
        .orderBy("partition")
    )
    sdf.show(20)
    return sdf

In [20]:
def avg_bytes_rows_partition(sdf):
    sdf = (
        sdf.select(f.mean("numFiles"), f.mean("compressedBytes"), f.mean("rows"))
        .withColumn("avg(compressedMB)", f.round(f.col("avg(compressedBytes)")/1024/1204, 1))
        .select("avg(numFiles)", "avg(compressedBytes)", "avg(compressedMB)", "avg(rows)")
    )
    sdf.show()

In [21]:
def file_analysis(path, num_files):
    file_size = get_parquet_file_size(path)
    avg_file_size = file_size/num_files
    print(" ")
    print("******** FILE SIZE ANALYSIS ********")
    print(f"File Size: {round(file_size/1024/1024, 1)} MB or {file_size} bytes")
    print(f"Num files: {num_files}")
    print(f"Avg file Size: {round(avg_file_size/1024/1024, 1)} MB or {avg_file_size} bytes")
    


In [22]:
def row_count_analysis(num_files, num_rows):
    print(" ")
    print("******** ROW COUNT ANALYSIS ********")    
    print(f"Num files written: {num_files}")
    print(f"Num rows written: {num_rows}")
    print(f"Num rows per file: {int(num_rows/num_files)}")

In [23]:
def get_actual_num_partitions(path):
    sdf = spark.read.parquet(path)
    print(" ")
    print("******** ACTUAL RESULTS ********")   
    print(f"ActualNumPartitions: {sdf.rdd.getNumPartitions()}")
    results_dict["ActualNumPartitions"] = sdf.rdd.getNumPartitions()


In [24]:
def noop_write(path):
    sdf = spark.read.parquet(path)
    sc.setJobDescription("WRITE")
    start_time = time.time()
    sdf.write.format("noop").mode("overwrite").save()
    end_time = time.time()
    sc.setJobDescription("None")
    duration = round(end_time - start_time, 2)
    results_dict["ExecutionTime"] = duration
    print(f"Duration: {duration} sec")

# 1. What influences the no. of partitions when loading parquet files
- Num Cores
- File Size
- Num parquet files
- Num of blocks/rowgroups within a parquet file
- Max Partition Size
- Max Cost Per Bytes

# 2. First simple experiment
- Experiment 1: 4 files, a 64 MB
- Experiment 2: 8 files, a 64 MB (128 MB; 140 MB max size)
- Experiment 3: 8 files, a 50 MB

Set-Up:
- 

In [80]:
num_files = 4
num_rows = 32000000
path = write_generator(num_rows, num_files)

Num partitions written: 4
Saved Path: D:/Spark/Data/4_files_32000000_rows.parquet


In [83]:
file_analysis(path, num_files)
row_count_analysis(num_files, num_rows)
set_configs(maxPartitionsMB=50, openCostInMB=4, minPartitions=4)
size = get_parquet_file_size(path)
estimate_num_partitions(size, num_files)
get_actual_num_partitions(path)
noop_write(path)
bytes_rows_per_partition(path)

 
******** FILE SIZE ANALYSIS ********
File Size: 259.3 MB or 271863691 bytes
Num files: 4
Avg file Size: 64.8 MB or 67965922.75 bytes
 
******** ROW COUNT ANALYSIS ********
Num files written: 4
Num rows written: 32000000
Num rows per file: 8000000
 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 50 MB or 52428800 bytes
OpenCostInBytes 4 MB or 4194304 bytes
Min Partitions: 4
 
******** ESTIMATION OF MAX SPLIT PARTITION BYTES AND NO PARTITIONS ********
Avg file Size Padded: 68.8 MB or 72160226.75 bytes
Padded File Size: 275.3 MB or 288640907 bytes
SizePerCore: 64.8 MB or 67965922.75 bytes
SizePerCorePadded: 68.8 MB or 72160226.75 bytes
MaxSplitPartitionBytes: 50.0 MB or 52428800 bytes
MaxFilesPerPartition 1
EstimatedPartitions: 4, unrounded: 4.0
EstimatedPartitionsInternet: 6, unrounded: 5.505388393402099
 
******** ACTUAL RESULTS ********
ActualNumPartitions: 6
Duration: 2.29 sec
+---------+--------+---------------+------------+-------+
|partition|numFiles|compressedBytes|comp

DataFrame[partition: int, numFiles: bigint, compressedBytes: bigint, compressedMB: double, rows: bigint]

In [84]:
num_files = 8
num_rows = 64000000
path = write_generator(num_rows, num_files)

Num partitions written: 8
Saved Path: D:/Spark/Data/8_files_64000000_rows.parquet


In [91]:
file_analysis(path, num_files)
row_count_analysis(num_files, num_rows)
set_configs(maxPartitionsMB=60, openCostInMB=4, minPartitions=4)
size = get_parquet_file_size(path)
estimate_num_partitions(size, num_files)
get_actual_num_partitions(path)
noop_write(path)
bytes_rows_per_partition(path)

 
******** FILE SIZE ANALYSIS ********
File Size: 518.9 MB or 544117207 bytes
Num files: 8
Avg file Size: 64.9 MB or 68014650.875 bytes
 
******** ROW COUNT ANALYSIS ********
Num files written: 8
Num rows written: 64000000
Num rows per file: 8000000
 
******** SPARK CONFIGURATIONS ********
MaxPartitionSize 60 MB or 62914560 bytes
OpenCostInBytes 4 MB or 4194304 bytes
Min Partitions: 4
 
******** ESTIMATION OF MAX SPLIT PARTITION BYTES AND NO PARTITIONS ********
Avg file Size Padded: 68.9 MB or 72208954.875 bytes
Padded File Size: 550.9 MB or 577671639 bytes
SizePerCore: 129.7 MB or 136029301.75 bytes
SizePerCorePadded: 137.7 MB or 144417909.75 bytes
MaxSplitPartitionBytes: 60.0 MB or 62914560 bytes
MaxFilesPerPartition 1
EstimatedPartitions: 8, unrounded: 8.0
EstimatedPartitionsInternet: 10, unrounded: 9.181843423843384
 
******** ACTUAL RESULTS ********
ActualNumPartitions: 10
Duration: 4.51 sec
+---------+--------+---------------+------------+-------+
|partition|numFiles|compressedBy

DataFrame[partition: int, numFiles: bigint, compressedBytes: bigint, compressedMB: double, rows: bigint]